In [ ]:
import numpy as np
from pulp import *
import random
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Define criteria values
criterion1_values = [0, 25, 50, 75, 100]
criterion2_values = ["Bad", "Poor", "Neutral", "Good", "Excellent"]

# Create LP problem
prob = LpProblem("Simplified_UTA_Method", LpMaximize)

# Create decision variables for criteria
criterion1_vars = {val: LpVariable(f"criterion1_{val}", lowBound=0, upBound=1) for val in criterion1_values}
criterion2_vars = {val: LpVariable(f"criterion2_{val}", lowBound=0, upBound=1) for val in criterion2_values}

# Monotonicity constraints
for i in range(len(criterion1_values) - 1):
    prob += criterion1_vars[criterion1_values[i]] <= criterion1_vars[criterion1_values[i + 1]], f"Monotonicity_C1_{i}"
for i in range(len(criterion2_values) - 1):
    prob += criterion2_vars[criterion2_values[i]] <= criterion2_vars[criterion2_values[i + 1]], f"Monotonicity_C2_{i}"

# Normalization constraints
prob += criterion1_vars[0] == 0, "Normalize_C1_Lowest"
prob += criterion2_vars["Bad"] == 0, "Normalize_C2_Lowest"
prob += criterion1_vars[100] + criterion2_vars["Excellent"] == 1, "Normalize_Criteria_Sum"

# Weight constraints (not part of UTA but additional constraint)
prob += criterion1_vars[100] <= 0.75, "Weight_C1_Limit"
prob += criterion2_vars["Excellent"] <= 0.75, "Weight_C2_Limit"

# Define alternatives with random evaluations
alternatives = {
    f"Alternative{i+1}": {
        "C1": random.choice(criterion1_values),
        "C2": random.choice(criterion2_values)
    }
    for i in range(4)
}

# Create utility variables and constraints
alternative_utilities = {}
for name, evals in alternatives.items():
    utility_var = LpVariable(f"{name}_Utility", lowBound=0)
    alternative_utilities[name] = utility_var

    # Sum of selected criteria variables defines utility
    prob += utility_var == lpSum([criterion1_vars[evals["C1"]], criterion2_vars[evals["C2"]]]), f"Utility_{name}"

# Preference constraints provided by the decision maker
epsilon = 0.01 # Small value to ensure strict preference
prob += alternative_utilities["Alternative1"] >= alternative_utilities["Alternative2"] + epsilon, "Pref_A1_A2"
prob += alternative_utilities["Alternative1"] >= alternative_utilities["Alternative3"] + epsilon, "Pref_A1_A3"
prob += alternative_utilities["Alternative2"] >= alternative_utilities["Alternative4"] + epsilon, "Pref_A2_A4"

# No specific objective function; checking feasibility
# You need to write your own objective function depending on the variant of the UTA method.

prob += 0 # Placeholder objective function

# Solve the problem
prob.solve()

# Print results
print(f"Status: {LpStatus[prob.status]}")
print("\nOptimal Solution:")
for var in prob.variables():
    print(f"{var.name} = {value(var)}")

print("\nObjective value:", value(prob.objective))
# In the results below, there is  __dummy = None which is a placeholder for the objective function.

In [ ]:
# Definicja słownika kryteriów
criteria = {
    "C1": (criterion1_values, criterion1_vars),
    "C2": (criterion2_values, criterion2_vars)
}

# Generowanie wykresów dla każdego kryterium
plt.figure(figsize=(12, 5))
for i, (criterion_name, (values, variables)) in enumerate(criteria.items(), 1):
    solver_values = [value(variables[val]) for val in values]

    plt.subplot(1, len(criteria), i)
    plt.plot(values, solver_values, marker='o', linestyle='-', label=f'$u(g_{{{criterion_name}}})$')
    plt.xlabel(f'$g_{{{criterion_name}}}$')
    plt.ylabel(f'$u(g_{{{criterion_name}}})$')
    plt.title(f'Funkcja użyteczności dla {criterion_name}')
    plt.xticks(values)
    plt.grid(True)
    plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Wczytanie danych
file_path = "../data.csv"
data = pd.read_csv(file_path, index_col=0)
criteria = list(data.columns)

# Tworzenie problemu optymalizacyjnego
prob = LpProblem("UTA_Method", LpMaximize)

# Tworzenie zmiennych decyzyjnych dla każdego kryterium
criteria_vars = {c: {val: LpVariable(f"{c}_{val}", lowBound=0, upBound=1) for val in sorted(data[c].unique())} for c in criteria}

# Ograniczenia monotoniczności
for c, vars_dict in criteria_vars.items():
    sorted_values = sorted(vars_dict.keys())
    for i in range(len(sorted_values) - 1):
        prob += vars_dict[sorted_values[i]] <= vars_dict[sorted_values[i + 1]], f"Monotonicity_{c}_{i}"

# Ograniczenia normalizacji
for c in criteria:
    prob += criteria_vars[c][min(data[c])] == 0, f"Normalize_{c}_Lowest"
prob += lpSum([criteria_vars[c][max(data[c])] for c in criteria]) == 1, "Normalize_Criteria_Sum"

# Tworzenie zmiennych użytkowych dla alternatyw
table_utilities = {}
for idx, row in data.iterrows():
    utility_var = LpVariable(f"Alt{idx}_Utility", lowBound=0)
    table_utilities[idx] = utility_var
    prob += utility_var == lpSum([criteria_vars[c][row[c]] for c in criteria]), f"Utility_Alt{idx}"

# Definiowanie preferencji
epsilon = 0.01
prob += table_utilities[1] >= table_utilities[2] + epsilon, "Pref_1_2"
prob += table_utilities[1] >= table_utilities[3] + epsilon, "Pref_1_3"
prob += table_utilities[2] >= table_utilities[4] + epsilon, "Pref_2_4"

# Brak określonej funkcji celu – tylko sprawdzanie wykonalności
prob += 0

# Rozwiązanie problemu
prob.solve()

# Wyświetlenie wyników
print(f"Status: {LpStatus[prob.status]}")
print("\nOptymalne rozwiązanie:")
for var in prob.variables():
    print(f"{var.name} = {value(var)}")
print("\nWartość funkcji celu:", value(prob.objective))

# Generowanie wykresów dla każdego kryterium
plt.figure(figsize=(12, 8))
for i, c in enumerate(criteria, 1):
    values = sorted(criteria_vars[c].keys())
    solver_values = [value(criteria_vars[c][v]) for v in values]

    plt.subplot(2, 2, i)
    plt.plot(values, solver_values, marker='o', linestyle='-', label=f'$u(g_{{{c}}})$')
    plt.xlabel(f'$g_{{{c}}}$')
    plt.ylabel(f'$u(g_{{{c}}})$')
    plt.title(f'Funkcja użyteczności dla {c}')
    plt.xticks(values, rotation=45)
    plt.grid(True)
    plt.legend()

plt.tight_layout()
plt.show()
